## Implementation

In [1]:

import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('C:/Users/Admin/Documents/ML-CP/Inventory-Demand-Foecasting-3/final_sales_data.csv')
#change path

In [3]:
df.columns

Index(['Unnamed: 0', 'id', 'item_id', 'dept_id', 'cat_id', 'store_id',
       'state_id', 'd', 'sold', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'sold_lag_1',
       'sold_lag_2', 'sold_lag_4', 'sold_lag_8', 'sold_lag_16', 'sold_lag_32'],
      dtype='object')

In [4]:
df.drop(['Unnamed: 0','id'],inplace=True,axis=1)

In [5]:
df.shape

(79857, 26)

In [6]:
df.head()

,item_id,dept_id,cat_id,store_id,state_id,d,sold,wm_yr_wk,weekday,wday,...,snap_CA,snap_TX,snap_WI,sell_price,sold_lag_1,sold_lag_2,sold_lag_4,sold_lag_8,sold_lag_16,sold_lag_32
0,0,0,0,0,0,516,1,11222,6,5,...,0,0,0,2.00,1.0,0.0,0.0,0.0,2.0,1.0
1,0,0,0,0,0,164,0,11124,1,3,...,0,1,1,2.00,0.0,2.0,0.0,0.0,1.0,4.0
2,0,0,0,0,0,1122,0,11404,3,2,...,0,0,0,2.24,2.0,0.0,0.0,0.0,2.0,0.0
3,0,0,0,0,0,756,0,11304,0,7,...,0,0,0,2.24,3.0,0.0,0.0,0.0,1.0,0.0
4,0,0,0,0,0,1690,0,11533,1,3,...,0,0,1,2.24,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor  # or other appropriate model
from sklearn.metrics import mean_absolute_error  # or other appropriate metric
from datetime import datetime

In [8]:
data=df
features = ['item_id', 'dept_id', 'cat_id', 'store_id',
            'state_id', 'd',   'wday', 'month', 'year',
            'sell_price', 
         'sold_lag_1','sold_lag_2', 'sold_lag_4',
            'sold_lag_8'
       #'sold_lag_16', 'sold_lag_32'
        ]
target = 'sold' 
X = data[features]
y = data[target]

In [9]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

### Random Forest Regressor

In [10]:
model = RandomForestRegressor(n_estimators=100)  # You can choose an appropriate model and hyperparameters
model.fit(train_data[features], train_data[target])

RandomForestRegressor()

In [11]:
val_predictions = model.predict(val_data[features])
val_mae = mean_absolute_error(val_data[target], val_predictions)
print(f"Validation MAE: {val_mae}")

Validation MAE: 1.121657901327323


### XGBOOST

In [12]:
# XGBoost Model
import xgboost as xgb
xgb_model = xgb.XGBRegressor()
xgb_model.fit(train_data[features], train_data[target])
xgb_predictions = xgb_model.predict(val_data[features])
val_mae2 = mean_absolute_error(val_data[target], xgb_predictions)
print(f"Validation MAE: {val_mae2}")


Validation MAE: 1.0991317426771854


### LightGBM

In [13]:
import lightgbm as lgb
# LightGBM Model
lgb_model = lgb.LGBMRegressor()
lgb_model.fit(train_data[features], train_data[target])
lgb_predictions = lgb_model.predict(val_data[features])
val_mae3 = mean_absolute_error(val_data[target], lgb_predictions)
print(f"Validation MAE: {val_mae3}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004733 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1088
[LightGBM] [Info] Number of data points in the train set: 63885, number of used features: 14
[LightGBM] [Info] Start training from score 1.412836
Validation MAE: 1.0949120355006075


### Catboost

In [14]:
import catboost as cb
# CatBoost Model
cb_model = cb.CatBoostRegressor(iterations=1000, depth=6, learning_rate=0.1)
cb_model.fit(train_data[features], train_data[target])
cb_predictions = cb_model.predict(val_data[features])
val_mae4 = mean_absolute_error(val_data[target], cb_predictions)
print(f"Validation MAE: {val_mae4}")

0:	learn: 4.2084110	total: 183ms	remaining: 3m 2s
1:	learn: 4.0054074	total: 219ms	remaining: 1m 49s
2:	learn: 3.8320201	total: 254ms	remaining: 1m 24s
3:	learn: 3.6665897	total: 295ms	remaining: 1m 13s
4:	learn: 3.5239619	total: 346ms	remaining: 1m 8s
5:	learn: 3.4056906	total: 462ms	remaining: 1m 16s
6:	learn: 3.2922754	total: 545ms	remaining: 1m 17s
7:	learn: 3.1949210	total: 580ms	remaining: 1m 11s
8:	learn: 3.1053666	total: 617ms	remaining: 1m 7s
9:	learn: 3.0262977	total: 652ms	remaining: 1m 4s
10:	learn: 2.9591546	total: 692ms	remaining: 1m 2s
11:	learn: 2.9450504	total: 740ms	remaining: 1m
12:	learn: 2.9304775	total: 769ms	remaining: 58.4s
13:	learn: 2.8664957	total: 805ms	remaining: 56.7s
14:	learn: 2.8531278	total: 825ms	remaining: 54.2s
15:	learn: 2.8393051	total: 867ms	remaining: 53.3s
16:	learn: 2.8261431	total: 926ms	remaining: 53.5s
17:	learn: 2.7731714	total: 962ms	remaining: 52.5s
18:	learn: 2.7615597	total: 999ms	remaining: 51.6s
19:	learn: 2.7126079	total: 1.04s	rema

### Ensemble Learning- Cross Validation Stacking

In [15]:

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

In [16]:
# Assuming you have a DataFrame called 'data' with your dataset
X_features = ['item_id', 'dept_id', 'cat_id', 'store_id',
            'state_id', 'd',   'wday', 'month', 'year',
            'sell_price', 
         'sold_lag_1','sold_lag_2', 'sold_lag_4',
            'sold_lag_8']
       
y_target = 'sold' 
X = data[X_features]
y = data[y_target]


In [17]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42)


In [18]:
base_models = [
    ('RandomForest', RandomForestRegressor(random_state=42)),
    ('GradientBoosting', GradientBoostingRegressor(random_state=42)),
    ('XGBoost', xgb.XGBRegressor(random_state=42)),
    ('LightGBM', lgb.LGBMRegressor(random_state=42)),
    ('CatBoost', cb.CatBoostRegressor(random_state=42, verbose=0)),
]


In [19]:
base_models_predictions = np.zeros((len(X), len(base_models)))



In [20]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, val_idx in kf.split(X):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    for i, (name, model) in enumerate(base_models):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        base_models_predictions[val_idx, i] = y_pred


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030742 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1088
[LightGBM] [Info] Number of data points in the train set: 63885, number of used features: 14
[LightGBM] [Info] Start training from score 1.412836
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017193 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1090
[LightGBM] [Info] Number of data points in the train set: 63885, number of used features: 14
[LightGBM] [Info] Start training from score 1.425170
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010965 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bi

In [21]:
meta_model = LinearRegression()
meta_model.fit(base_models_predictions, y)
final_predictions = meta_model.predict(base_models_predictions)


In [22]:
mae_stacked = mean_absolute_error(y, final_predictions)

print("Mean Absolute Error (MAE) for the stacked model:", mae_stacked)

Mean Absolute Error (MAE) for the stacked model: 1.0689935624044735


In [23]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor

# # Assuming you have a DataFrame called 'data' with your dataset
# X = data[[ 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'event_name_1',
#           'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX',
#           'snap_WI', 'sell_price', 'sold_lag_1', 'sold_lag_2', 'sold_lag_4',
#           'sold_lag_8', 'sold_lag_16', 'sold_lag_32']]
# y = data['sold']

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define your model (Random Forest Regressor in this example)
# model_rf = RandomForestRegressor(n_estimators=100, random_state=42)

# # Fit the model to your training data
# model_rf.fit(X_train, y_train)

# # Get feature importances
# feature_importances = model_rf.feature_importances_

# # Create a DataFrame to display feature names and their importances
# importances_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

# # Sort the DataFrame by importance in descending order
# importances_df = importances_df.sort_values(by='Importance', ascending=True)

# # Display the top N important features
# N = 26  # You can change N to display more or fewer features
# top_features = importances_df.head(N)
# print(top_features)


## Prediction on sample data

In [30]:
item_id=5
dept_id =5
cat_id = 3
store_id = 4
state_id=0
d=28
wday=5
month = 5
year=2023
sell_price=15.12
sold_lag_1=215.2
sold_lag_2=120.5
sold_lag_4=172.5
sold_lag_8=245.6


input_data = {
    'item_id':item_id,
    'dept_id': dept_id,
    'cat_id': cat_id,
    'store_id': store_id,
    'state_id': state_id,  # if applicable
    'd':d,
    'wday':wday,
    'month': month,
    'year': year,
    'sell_price':sell_price,
    'sold_lag_1':sold_lag_1,
    'sold_lag_2':sold_lag_2,
    'sold_lag_4':sold_lag_4,
    'sold_lag_8':sold_lag_8
    # Add other features here as needed
}

safety_stock=20
import math
input_df = pd.DataFrame([input_data])

# Use the trained model to make predictions for future sales
# Exclude the features not provided in the input
input_features = [feature for feature in features if feature in input_df.columns]
predicted_sales = model.predict(input_df[input_features])[0]
predicted_sales = math.ceil(predicted_sales)

# Calculate inventory needed (customize this based on your business logic)
# For example, if you have a simple rule like "inventory_needed = 2 * predicted_sales," you can do:
inventory_needed = predicted_sales + safety_stock

# Print the results
print(f"Predicted Sales for month {month} : {predicted_sales} units")
print(f"Recommended Inventory Needed for month {month} : {inventory_needed} units")



Predicted Sales for month 5 : 228 units
Recommended Inventory Needed for month 5 : 248 units
